## 多元高斯分布的EM算法实现

因为要用到GMM的EM算法，所以自己写了一个（本人很菜，这一点代码写了两天）。写完发现sklearn的包里有。。那不如检测一下自己写的对不对吧。嘿嘿。做了下实现，用同一组参数初始化，看下来是没问题的。

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
import numpy as np
from numpy import linalg
import math
from sklearn.mixture import GaussianMixture


class GMM(object):
    def __init__(self, X, K):
        self.N, self.D = X.shape
        self.K = K
        self.Mu, self.Sigma, self.Pi, self.Gamma = self.init_params()
    
    def init_params(self):
        Mu = np.random.rand(self.K, self.D)  # K*D
        Sigma = np.array([np.eye(self.D)] * self.K) * 0.1  # K*D*D
        Pi = np.array([1.0 / self.K] * self.K)  # K
        Gamma = np.random.rand(self.N, self.K)
        return Mu, Sigma, Pi, Gamma
    
    def e_step(self, X):
        prob = []
        for i in range(self.K):
            prob.append(get_Gaussian_prob(X, self.Mu[i], self.Sigma[i]))
        prob = np.array(prob).T
        Pi_Gussian = prob * self.Pi
        sum_K = np.sum(Pi_Gussian, axis=1)
        return (Pi_Gussian.T / sum_K).T  # Gamma
    
    def m_step(self, X):
        NK = np.sum(self.Gamma, axis=0)
        _1_NK = 1 / NK
        # get new Mu
        m = np.dot(self.Gamma.T, X)
        newMu = (_1_NK * m.T).T
        # get new Sigma
        newSigma = []
        for i in range(self.K):
            newSigma.append(_1_NK[i] * np.dot((self.Gamma[:, i] * (X - newMu[i]).T), (X - newMu[i])))
        newSigma = np.array(newSigma)
        # get Pi
        newPi = NK / self.N
        return newMu, newSigma, newPi
    
    def train(self, X, times):
        for i in range(times):
            self.Gamma = self.e_step(X)
            self.Mu, self.Sigma, self.Pi = self.m_step(X)
        return self.Mu, self.Sigma, self.Pi, self.Gamma


def get_Gaussian_prob(X, mu, sigma):
    """已知一个高斯分布的参数，求其概率"""
    D = len(sigma)
    if X.size == D:
        n = 1
    else:
        n, _ = X.shape
    det = linalg.det(sigma)
    
    norm_const = 1.0 / (math.pow(2 * math.pi, float(D) / 2) * math.pow(det, 1.0 / 2))
    x_mu = np.array(X - mu)
    inv = linalg.inv(sigma)
    result = np.power(math.e, -0.5 * np.dot(np.dot(x_mu, inv), x_mu.T))
    if n != 1:
        result = np.diag(result)
    return norm_const * result


In [3]:
K,D,n = 2,3,5

In [4]:
def init_params(K, D):
    Mu = np.random.rand(K, D)  # K*D
    Sigma = np.array([np.eye(D)] * K) * 0.1  # K*D*D
    Pi = np.array([1.0 / K] * K)  # K
    return Mu, Sigma, Pi

Mu, Sigma, Pi = init_params(K,D)
X = np.random.rand(n, D)


In [5]:
Mu
Sigma
Pi
X

array([[0.19571869, 0.04502597, 0.55998468],
       [0.37129604, 0.09157221, 0.87149954]])

array([[[0.1, 0. , 0. ],
        [0. , 0.1, 0. ],
        [0. , 0. , 0.1]],

       [[0.1, 0. , 0. ],
        [0. , 0.1, 0. ],
        [0. , 0. , 0.1]]])

array([0.5, 0.5])

array([[0.02139303, 0.7811423 , 0.48241082],
       [0.79871001, 0.15815766, 0.43721512],
       [0.63534214, 0.43237055, 0.13972686],
       [0.03113701, 0.7675715 , 0.86453491],
       [0.98713439, 0.30671128, 0.18768251]])

In [6]:
# 自己写的GMM
gmmzj = GMM(X,K)
gmmzj.Mu, gmmzj.Sigma, gmmzj.Pi = Mu, Sigma, Pi

In [7]:
# sklearn的包
gmmsk = GaussianMixture(n_components=K, weights_init=Pi, means_init=Mu, precisions_init=linalg.inv(Sigma), max_iter=2)

In [8]:
# 迭代两次
gmmzj.train(X,2)

(array([[0.49494183, 0.50316971, 0.36650481],
        [0.49447294, 0.47015159, 0.49832464]]),
 array([[[ 0.15608295, -0.08766777, -0.06588976],
         [-0.08766777,  0.05471829,  0.03240687],
         [-0.06588976,  0.03240687,  0.05291524]],
 
        [[ 0.16231338, -0.10061026, -0.09171623],
         [-0.10061026,  0.07074169,  0.05029592],
         [-0.09171623,  0.05029592,  0.07583287]]]),
 array([0.5766249, 0.4233751]),
 array([[0.79573658, 0.20426342],
        [0.3826533 , 0.6173467 ],
        [0.75416763, 0.24583237],
        [0.3275795 , 0.6724205 ],
        [0.62298746, 0.37701254]]))

In [9]:
# 迭代两次
gmmsk.fit(X)

gmmsk.means_
gmmsk.covariances_
gmmsk.weights_

/home/carbondata/.pyenv/versions/3.6.5/envs/xyl/lib/python3.6/site-packages/sklearn/mixture/base.py:273: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=2,
        means_init=array([[0.19572, 0.04503, 0.55998],
       [0.3713 , 0.09157, 0.8715 ]]),
        n_components=2, n_init=1,
        precisions_init=array([[[10.,  0.,  0.],
        [ 0., 10.,  0.],
        [ 0.,  0., 10.]],

       [[10.,  0.,  0.],
        [ 0., 10.,  0.],
        [ 0.,  0., 10.]]]),
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False,
        weights_init=array([0.5, 0.5]))

array([[0.49494685, 0.50316453, 0.36650908],
       [0.49446609, 0.47015811, 0.49832092]])

array([[[ 0.15608373, -0.08766827, -0.06589002],
        [-0.08766827,  0.0547203 ,  0.03240688],
        [-0.06589002,  0.03240688,  0.05291725]],

       [[ 0.16231478, -0.10061   , -0.09171539],
        [-0.10061   ,  0.07074202,  0.05029515],
        [-0.09171539,  0.05029515,  0.07583411]]])

array([0.57663165, 0.42336835])